In [11]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-
'''
@Author: zhiwei.xu
@LastEditors: zhiwei.xu
@Date: 2019-05-13 16:22:47
@LastEditTime: 2019-05-13 17:37:56
'''
import requests

BAIDU_AK = "duzy3ZKbuSARjof2clCzz3m1SAAKOaDe"


class BaiduPOI(object):
    def __init__(self, tag, loc):
        self.tag = tag
        self.loc = loc
        self.ak = BAIDU_AK
        self.urls = self._urls()

    def _urls(self):
        urls = []
        for page in range(0, 20):
            url = f"http://api.map.baidu.com/place/v2/search?query={self.tag}&bounds={self.loc}&page_size=20&page_num={page}&output=json&ak={self.ak}"
            urls.append(url)
        return urls

    def search(self):
        json_sel = []
        for url in self.urls:
            resp = requests.get(url)
            data = resp.json()
            result = data.get('results', [])
            if not data:
                break
            for item in data['results']:
                # import pdb; pdb.set_trace()
                jname = item["name"]
                jlat = item["location"]["lat"]
                jlng = item["location"]["lng"]
                address = item["address"]
                tel = item.get("telephone", "")
                js_sel = jname + ',' + str(jlat) + ',' + str(jlng) + ',' + str(tel) + ',' + str(address)
                json_sel.append(js_sel)
        return json_sel


class LocaDiv(object):
    """
    在纬度相等的情况下：
        经度每隔0.00001度，距离相差约1米；
        经度每隔0.0001度，距离相差约10米；
        经度每隔0.001度，距离相差约100米；
        经度每隔0.01度，距离相差约1000米；
        经度每隔0.1度，距离相差约10000米。
    在经度相等的情况下：
        纬度每隔0.00001度，距离相差约1.1米；
        纬度每隔0.0001度，距离相差约11米；
        纬度每隔0.001度，距离相差约111米；
        纬度每隔0.01度，距离相差约1113米；
        纬度每隔0.1度，距离相差约11132米。
    """
    def __init__(self, loc_all):
        self.loc_all = loc_all.split(',')
        self.lat_all = self._lat_all()
        self.lng_all = self._lng_all()

    def _lat_all(self,size=0.02):
        """size为0.02 约2.2km"""
        lat_sw = float(self.loc_all[0])
        lat_ne = float(self.loc_all[2])
        lat_list = []
        for i in range(0, int((lat_ne - lat_sw + size) / size)):  # 网格
            lat_list.append(lat_sw + size * i)
        lat_list.append(lat_ne)
        print(len(lat_list),lat_list)
        return lat_list

    def _lng_all(self,size=0.02):
        """size为0.02 约2km"""
        lng_sw = float(self.loc_all[1])
        lng_ne = float(self.loc_all[3])
        lng_list = []
        for i in range(0, int((lng_ne - lng_sw + size) / size)):  # 网格
            lng_list.append(lng_sw + size * i)
        lng_list.append(lng_ne)
        print(len(lng_list))
        return lng_list

    def ls_dot(self):
        row = []
        for lat in self.lat_all:
            tmp_row = []
            for lng in self.lng_all:
                tmp_row.append(f"{lat},{lng}")
            row.append(tmp_row)
        return row

    def ls_row(self):
        l2 = self.lng_all
        dots = self.ls_dot()
#         print(dots)
        loc = []
        for i in range(0, len(self.lat_all) - 1):
            for j in range(0, len(self.lng_all) - 1):
                loc.append(f"{dots[i][j]},{dots[i+1][j+1]}")
        print(len(loc))
        return loc
a = LocaDiv('39.7022901558,116.1115072963,40.1489353260,116.6978758455').ls_row()
"39.7040730000,116.1003230000,40.1830090000,116.6980920000"

24
31
690


In [13]:
# 计算经纬度距离

from math import radians, cos, sin, asin, sqrt
 
def haversine(lat1, lon1, lat2, lon2): # 维度1，经度1，维度2，经度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    # haversine公式
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    return c * r * 1000

haversine(39.7022901558,116.1115072963,40.1489353260,116.6978758455)

70474.61402861307

In [31]:
import numpy as np

lst = [2, 2, 2, 1, 2, 2, 1, 2, 2, 3, 2, 3, 3, 4, 2, 2, 2, 2, 3, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 6, 13, 6, 6, 2, 2, 2, 2, 3, 2, 2, 13, 5, 2, 2, 2, 3, 3, 2, 1, 1, 2, 4, 3, 1, 2, 1, 1, 2, 1, 2, 6, 5, 3, 2, 4, 1, 2, 2, 5, 5, 13, 4, 3, 2, 2, 3, 2, 1, 2, 2, 4, 5, 2, 2, 2, 2, 3, 1, 1, 2, 2, 4, 3, 4, 4, 2, 2, 2, 2, 4, 10, 4, 4, 2, 3, 2, 2, 2, 3, 2, 5, 9, 2, 2, 3, 1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 6, 4, 11, 3, 3, 2, 3, 2, 2, 2, 7, 5, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 6, 3, 3, 2, 1, 1, 2, 3, 1, 5, 3, 5, 4, 5, 6, 10, 2, 5, 3, 4, 3, 6, 2, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 3, 2, 2, 3, 6, 4, 5, 2, 3, 3, 3, 4, 4, 3, 4, 7, 2, 2, 2, 3, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3, 2, 4, 8, 9, 6, 5, 12, 13, 10, 9, 7, 12, 4, 8, 7, 3, 3, 3, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 5, 6, 9, 7,3, 7, 9, 2, 11, 12, 18, 12, 7, 6, 8, 3, 1, 2, 2, 2, 4, 4, 2, 2, 2, 1, 2, 2, 2, 5, 9, 10, 6, 14, 15, 13, 12, 16, 8, 10, 19, 19, 8, 3, 3, 2, 2, 3, 2, 7, 10, 11, 4, 4, 3, 2, 4, 7, 9, 7, 8, 10, 8, 10, 10, 14, 20, 10, 20, 16, 20, 20, 19, 9, 8, 10, 8, 7, 3, 13, 7, 5, 3, 7, 2, 4, 6, 10, 4, 3, 7, 1, 8, 7, 8, 14, 14, 8, 15, 20, 20, 13, 12, 20, 5, 4, 6, 12, 7, 5, 3, 4, 3, 4, 2, 2, 2, 3, 2, 5, 3, 6, 7, 13, 11, 15, 14, 20, 20, 17, 20, 11, 3, 4, 2, 6, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 2, 1, 2, 2, 3, 4, 11, 18, 17, 13, 13, 10, 16, 13, 14, 7, 14, 2, 4, 3, 3, 2, 2, 2, 2, 2, 2, 1, 1, 1,2, 4, 3, 2, 2, 2, 5, 20, 20, 11, 15, 7, 18, 7, 6, 20, 13, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 4, 4, 4, 6, 11, 7, 7, 11, 3, 6, 17, 9, 3, 5, 3, 8, 2, 3, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 6, 4, 9, 6, 7, 3, 2, 7, 6, 4, 4, 3, 3, 2, 3, 2, 2, 2, 4, 1, 3, 1, 2, 2, 2, 3, 3, 2, 2, 2, 3, 8, 7, 11, 4, 3, 3, 11, 7, 2, 3, 3, 3, 2, 2, 2, 7, 3, 2, 2, 3, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 2, 8, 9, 10, 3, 2, 14, 7, 2, 1, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 6, 7, 11, 6, 5, 4, 8, 7, 3, 2, 2, 2, 5, 2, 3, 3, 2, 1, 3, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 5,7, 3, 2, 3, 12, 5, 4, 2, 2, 1, 3, 3, 5, 4, 3, 1, 3, 2, 6, 3, 2, 1, 2, 2, 2, 2, 2, 2, 4, 5, 2, 1, 3, 2, 2, 3, 5, 4, 2, 2, 2, 2, 2, 4, 2, 2, 3, 5, 15, 3, 2, 1, 4, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1,1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 3, 3, 4, 1, 1]
print(len(lst), f'aver:{sum(lst)/len(lst)}')

dict(zip(*np.unique(lst, return_counts=True)))

690 aver:4.397101449275362


{1: 80,
 2: 265,
 3: 95,
 4: 52,
 5: 30,
 6: 26,
 7: 31,
 8: 16,
 9: 11,
 10: 14,
 11: 12,
 12: 8,
 13: 12,
 14: 8,
 15: 5,
 16: 3,
 17: 3,
 18: 3,
 19: 3,
 20: 13}

### 计算坐标是否在多边形内部（后续用来判定坐标点是否在国内）

In [93]:
def is_point_in_polygon(point, poly_list):
    '''
    :param point: tuple (lon, lat) 经纬度坐标
    :param poly_list: list [(lon, lat)...] 多边形点的顺序需根据顺时针或逆时针，不能乱
    '''
    # 多边形变数少于3，直接返回False
    p_cnt = len(poly_list)
    if(p_cnt < 3):
        return False
    
    # 判断是否在外包矩形内，如果不在，直接返回False
    alon = point[0]
    alat = point[1]
    lon_list = [] # 经度
    lat_list = [] # 纬度
    for poi in poly_list:
        lon_list.append(poi[0])
        lat_list.append(poi[1])
    lon_max = max(lon_list)
    lon_min = min(lon_list)
    lat_max = max(lat_list)
    lat_min = min(lat_list)
    if (alon > lon_max or alon < lon_min or alon > lat_max or alat < lat_min):
        return False
    
    # 判断点在多边形内
    count = 0
    for i in range(p_cnt):
        # 边第一点
        plon1 = poly_list[i][0]
        plat1 = poly_list[i][1]
        # 边第二点
        if(i == p_cnt - 1):
            plon2 = poly_list[0][0]
            plat2 = poly_list[0][1]
        else:
            plon2 = poly_list[i + 1][0]
            plat2 = poly_list[i + 1][1]
        
        # 点与多边形顶点重合,在顶点上
        if (alon == plon1 and alat == plat1) or (alon == plon2 and alat == plat2):
            return False
        
        # 判断线段两端点是否在判断点射线两侧，不在不相交，射线（-∞，lat）（lon, lat）
        if (plat1 <= alat and alat < plat2) or (plat2 <= alat and alat < plat1):
            # 求线段与射线交点的经度 再和判断点经度比较
            plon = plon2 - ((plat2 - alat) * (plon2 - plon1))/(plat2 - plat1)
            # 点在多边形边上
            if plon == alon:
                return False
            if (plon < alon):
                count += 1

    if count%2 == 0:
        return False
    else:
        return True

print(is_point_in_polygon([1.1,1], [[0,0],[1,0],[1,1],[0,1]]))

False


### 将中国分为几大块
- 纬度3.86~53.55,经度73.66~135.05
- 

In [ ]:
#

[
(49.220400,079.446200,42.889900,096.330000),
(54.141500,109.687200,39.374200,135.000200),
(42.889900,073.124600,29.529700,124.143255),
(29.529700,082.968400,26.718600,097.035200),
(29.529700,097.025300,20.414096,124.367395),
(20.414096,107.975793,17.871542,111.744104),
]

In [130]:
def n(n):
    a=0                     # 1
    b=1                     # 1
    for i in range(n):      # n
        s=a+b               # n 
        b=a                 # n
        a=s                 # n
    return a,b              # 1

def demo1(n):
    sum=0                   # 1
    for i in range(n):      # n
        for j in range(n):  # n^2
            sum += 1        # n^2
    return sum              # 1

def demo2(n):
    x=y=0                           # 1
    for i in range(1, n):           # n-1
        y = y+1                     # n-1
        for j in range(0, 2*n+1):   # (n-1)*(2*n+1)
            x += 1                  # (n-1)*(2*n+1)
    return x,y                      # 1
print(demo0(5),demo1(5))
# 25 (44, 4)

def demo3(n):
    i=1             # 1
    while (i<=n):   # 由循环体决定
        i=i*2       # 设频度是f(n),则：2^f(n)<=n;f(n)<=log2n
    return i        # 1

def demo4(n):
    x = 0                          # 1
    for i in range(n):             # n
        for j in range(0, i):      # (n-1)*n/2 >> n^2/2 
            for k in range(0, j):  # (1+(1+n)*n/2)*/2/2
                x += 2
    return x
demo4(4)

25 25


8

In [131]:
class Node(object) :
    """A node in a binary search tree."""
    
    def __init__(self, value) :
        """A new Node in a Binary Search Tree

        Parameters:
        value: Element to be stored in this Node.
        Must be comparable by ==, > and <.
        """
        self._value = value
        self._left = None
        self._right = None

    def insert(self, value) :
        """Add 'value' into this Node in the search tree.

        Parameters:
        value: Element to be stored in this Node.
        Must be comparable by ==, > and <.
        """
        if value < self._value :
            if self._left is None :
                self._left = Node(value)
            else :
                self._left.insert(value)
        else :
            if self._right is None :
                self._right = Node(value)
            else :
                self._right.insert(value)
    
    def to_list(self) :
        """Return a sorted list of the values of this Node's children."""
        result = []
        if self._left is not None :
            result.extend(self._left.to_list())
        result.append(self._value)
        if self._right is not None :
            result.extend(self._right.to_list())
        return result
    
    def __contains__(self, value) :
        if value == self._value :
            return True
        elif value < self._value and self._left is not None :
            return value in self._left
        elif value > self._value and self._right is not None :
            return value in self._right
        else :
            return False

    def __str__(self) :
        return "({0}, {1}, {2})".format(self._left, self._value, self._right)


In [138]:
class BinarySearchTree(object) :
    """A binary search tree."""
    
    def __init__(self) :
        """A new empty binary search tree."""
        self._root = None
    
    def insert(self, value) :
        """Add 'value' into the tree.

        Preconditions:
        'value' is comparable by at least the ==, > and < operators.
        """
        if self._root is None :
            self._root = Node(value)
        else :
            self._root.insert(value)
            
    def to_list(self) :
        """Return a sorted list of the values in this tree."""
        if self._root is None :
            return []
        return self._root.to_list()
        
    def __contains__(self, value) :
        return self._root is not None and value in self._root
        
    def __str__(self) :
        if self._root is None :
            return "()"
        return str(self._root)

def tree_sort(lst) :
    """A sort implementation using a tree.

    Parameters:
    lst: List of elements to be sorted.
    Return:
    Sorted list of elements.
    Preconditions:
    All elements of 'lst' are comparable by
    at least the ==, >, and < operators.
    """
    tree = BinarySearchTree()
    for element in lst :
        tree.insert(element)
    return tree.to_list()

In [139]:
tree = BinarySearchTree()
lst = [5, 3, 2, 4, 7, 6, 8]
for v in lst:
    tree.insert(v)
print(tree.to_list())  # 打印树里面的元素，可使用二叉树实现排序
print(7 in tree)       # 判断值是否在树内
print(7.5 in tree)     # 判断值是否在树内
print(tree)            # 打印数
print(tree_sort(lst))  # 利用二叉树进行排序
# [2, 3, 4, 5, 6, 7, 8]
# True
# False
# (((None, 2, None), 3, (None, 4, None)), 5, ((None, 6, None), 7, (None, 8, None)))
# [2, 3, 4, 5, 6, 7, 8]

[2, 3, 4, 5, 6, 7, 8]
True
False
(((None, 2, None), 3, (None, 4, None)), 5, ((None, 6, None), 7, (None, 8, None)))
[2, 3, 4, 5, 6, 7, 8]


In [142]:
def num_paths(x, y) :
    """Calculate the number of paths to point (x,y)

    Parameters:
    x (int): x coordinate of the point.
    y (int): y coordinate of the point.
    Return:
    int: Number of paths from (0,0) to (x,y).
    """
    if x == 0 or y == 0 :
        return 1
    else :
        return num_paths(x, y-1) + num_paths(x-1, y)

print(num_paths(7,7))
print(num_paths(1,1))
# 3432
# 2

3432
2


In [136]:
def factorial(n) :
    """Calculate the factorial of the given number 'n'.

    Parameters:
    n (int): Number for which the factorial is to be calculated.
    Return:
    int: Factorial of 'n'.
    """
    if n == 0 :
        return 1
    else :
        return factorial(n-1) * n
fa

[2, 3, 4, 5, 6, 7, 8]


In [145]:
import tkinter as tk
root = tk.Tk()
root.title('HelloWorld Example')
hw = tk.Label(root, text='Hello World!')
hw.pack()
root.mainloop()



TclError: no display name and no $DISPLAY environment variable

In [ ]:
import tkinter as tk
class EvalApp(object):
    """A simple application to allow a user to enter an and evaluate it.
    """
    def __init__(self, master):
        """Initialise the expression evaluator's application window.
        Parameters:
        master (Tk): Main window for application.
        """
        self._master = master
        master.title('Expression Evaluator')
        self._num = 0.0
        expressionLbl = tk.Label(master, text='Enter Expression: ')
        expressionLbl.pack(side=tk.LEFT)
        self._entry = tk.Entry(master, width=20)
        self._entry.pack(side=tk.LEFT)
        self._entry.insert(tk.END, str(self.num))
        evalBtn = tk.Button(master, text="Evaluate", command=self.evaluate)
        evalBtn.pack(side=tk.LEFT)
        self._result = tk.Label(master, text="", bg="grey")
        self._result.pack(side=tk.LEFT, padx=20)

    def evaluate(self):
        """Evaluates the expression in the Entry widget and
        displays the result in the result Label.
        """
        try:
            self._num = eval(self._entry.get())
            self._result.config(text="The Result is: {0}".format(self._num),
            bg="grey")
        except Exception as e:
            self._result.config(text="An error occurred: {0}".format(e), bg="red")
root = tk.Tk()
app = EvalApp(root)
root.mainloop()

In [150]:
(lambda x : lambda e : x+1)(1)

<function __main__.<lambda>.<locals>.<lambda>(e)>

In [ ]:
(lambda x: lambda e: self._activate_item(x))(i)

In [151]:
135555+1666904+388313+154297+601067+227507+342688+980226

4496557

In [152]:
135555+1666904+388313+154297+601067+227507+342688+980226+295702

4792259

In [153]:
6698466+641213

7339679

In [154]:
2560208+56629+44574+190711+73297+143532

3068951

In [155]:
import time
time.gmtime()

time.struct_time(tm_year=2019, tm_mon=5, tm_mday=27, tm_hour=2, tm_min=44, tm_sec=2, tm_wday=0, tm_yday=147, tm_isdst=0)

In [160]:
import datetime
import time
%timeit datetime.datetime.now().strftime("%Y%m%d%H%M%S")
%timeit time.strftime('%Y%m%d%H%M%S', time.localtime(time.time()))

3.98 µs ± 808 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.09 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [161]:
# 网格数
print(f"网格总数：{(236040+1828028+542136+293724+765700+504885+589248+1159275+522655+6775852+1047476+3017812+100440+113832+361054+124020+278860)/10**4}万")
grid = 15418560
print(f"国内网格：{grid/10**4}万")
# 网格获取citycode时间预估
print("\n网格获取citycode时间预估:")
print(f"单天处理：{(236040/1140)*60*60*24/10**4}万")
print(f"预计总耗时：{grid/((236040/1140)*60*60*24)}天")
# 获取POI时间预估
print("\n获取POI时间预估:")
total = grid*1.5*17
print(f"预计总请求：{total/10**8}亿（国内网格数*页数取中值1.5*分类数17）")  # 网格数*页数（取中间值）*分类数
per_day = 49.5*5*10000/195/60*86400
print(f"单机单天处理：{per_day/10**4}万（单机33进程）")  # 33进程
print(f"单台服务器总耗时：{total/(per_day)}天")  # 
print(f"三台服务器总耗时：{total/(per_day)/3}天")

网格总数：1826.1037万
国内网格：1541.856万

网格获取citycode时间预估:
单天处理：1788.9347368421052万
预计总耗时：0.8618849912444219天

获取POI时间预估:
预计总请求：3.9317328亿（国内网格数*页数取中值1.5*分类数17）
单机单天处理：1827.6923076923076万（单机33进程）
单台服务器总耗时：21.512006060606062天
三台服务器总耗时：7.170668686868687天


In [164]:
网格总数：15205179
已处理：3465000
效率：约300万/天
目前POI数：6922018（只保留有手机号poi，按号码&名称去重）


5.0

In [173]:
a = list(range(1,10))

In [174]:
a

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [175]:
a[1::2]

[2, 4, 6, 8]

In [176]:
a[::2]

[1, 3, 5, 7, 9]

In [183]:
grid = [13090040.000204807,4119455.9797931877,13091040.000204807,4120455.9797931877]
def _cut_grid(grid):
        lon = grid[::2]
        lat = grid[1::2]
        lon_half = sum(lon) / 2
        lat_half = sum(lat) / 2
        grids = [
            (lon[0], lat[0], lon_half, lat_half),
            (lon_half, lat[0], lon[1], lat_half),
            (lon[0], lat_half, lon_half, lat[1]),
            (lon_half, lat_half, lon[1], lat[1])]
        return grids
_cut_grid(grid)

[(13090040.000204807,
  4119455.9797931877,
  13090540.000204807,
  4119955.9797931877),
 (13090540.000204807,
  4119455.9797931877,
  13091040.000204807,
  4119955.9797931877),
 (13090040.000204807,
  4119955.9797931877,
  13090540.000204807,
  4120455.9797931877),
 (13090540.000204807,
  4119955.9797931877,
  13091040.000204807,
  4120455.9797931877)]

In [179]:
sum(lon)/2

116.4733495

In [186]:
(3711455.9797931877+3712455.9797931877)/2

3711955.9797931877

In [191]:
a ={1:2}
{**a, **{2:3}}

{1: 2, 2: 3}

In [194]:
a = '2019-05-24 13:53:14,014 [WARNING] __main__: data_over_url:https://map.baidu.com/?qt=spot&wd=购物&c=265&on_gel=1&b=(13152040.000204807,4810455.979793187;13153040.000204807,4811455.979793187)&rn=50&pn=0&t=1558677193784'
import re
pt = 'wd=(.*?)&c=(.*?)&on_gel=1&b=\((.*?)\)&rn'
re.findall(pt, a)

[('购物',
  '265',
  '13152040.000204807,4810455.979793187;13153040.000204807,4811455.979793187')]

In [199]:
19.8/(7*60+34)*60*24*3

188.40528634361237

In [198]:
1.65*24*2

79.19999999999999